In [6]:
%load_ext autoreload
%autoreload 2

# Logit PAG

GenerationMixin.generate 이 좀 어질어질하네.

되도록 건드리지 않는 것을 추구해보자

In [1]:
import transformers
import torch

model_id = "meta-llama/Meta-Llama-3-8B"

pipe = transformers.pipeline(
    "text-generation", model=model_id, model_kwargs={"torch_dtype": torch.float16}, device_map="cuda"
)
# pipe("Hey how are you doing today?", max_length=256, truncation=True)

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 4/4 [00:48<00:00, 12.24s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [2]:
# Adapted from LlamaAttention.forward
def perturb_attn_map(module, input, kwargs, output, guidance_scale: float=7.5):
    hidden_states = kwargs['hidden_states']
    attention_mask = kwargs['attention_mask']
    position_ids = kwargs['position_ids']
    past_key_value = kwargs['past_key_value']
    output_attentions = kwargs['output_attentions']
    use_cache = kwargs['use_cache']
    cache_position = kwargs['cache_position']

    if output_attentions:
        raise NotImplementedError()

    bsz, q_len, _ = hidden_states.size()

    query_states = module.q_proj(hidden_states)
    key_states = module.k_proj(hidden_states)
    value_states = module.v_proj(hidden_states)

    query_states = query_states.view(bsz, q_len, module.num_heads, module.head_dim).transpose(1, 2)
    key_states = key_states.view(bsz, q_len, module.num_key_value_heads, module.head_dim).transpose(1, 2)
    value_states = value_states.view(bsz, q_len, module.num_key_value_heads, module.head_dim).transpose(1, 2)

    cos, sin = module.rotary_emb(value_states, position_ids)
    query_states, key_states = apply_rotary_pos_emb(query_states, key_states, cos, sin)

    # In case static cache is used, it is an instance attribute.
    past_key_value = getattr(module, "past_key_value", past_key_value)

    if past_key_value is not None:
        # sin and cos are specific to RoPE models; cache_position needed for the static cache
        cache_kwargs = {"sin": sin, "cos": cos, "cache_position": cache_position}
        key_states, value_states = past_key_value.update(key_states, value_states, module.layer_idx, cache_kwargs)

    key_states = repeat_kv(key_states, module.num_key_value_groups)
    value_states = repeat_kv(value_states, module.num_key_value_groups)

    k_len = key_states.size(-2)
    causal_mask = attention_mask
    if attention_mask is not None:
        causal_mask = causal_mask[:, :, :, :k_len]

    # perturbed attn
    attention_mask = torch.zeros(k_len, k_len) + torch.eye(k_len)
    attention_mask = attention_mask.bool().to(query_states.device)
    attention_mask = attention_mask[None, None, :q_len, :k_len] # |b, head, l, l|

    if query_states.device.type == "cuda" and causal_mask is not None:
        query_states = query_states.contiguous()
        key_states = key_states.contiguous()
        value_states = value_states.contiguous()

    attn_output = torch.nn.functional.scaled_dot_product_attention(
        query_states,
        key_states,
        value_states,
        attn_mask=attention_mask,
        dropout_p=module.attention_dropout if module.training else 0.0,
        is_causal=causal_mask is None and q_len > 1,
    )

    attn_output = attn_output.transpose(1, 2).contiguous()
    attn_output = attn_output.view(bsz, q_len, module.hidden_size)

    attn_output = module.o_proj(attn_output)

    return attn_output, None, past_key_value

def apply_rotary_pos_emb(q, k, cos, sin, position_ids=None, unsqueeze_dim=1):
    """Applies Rotary Position Embedding to the query and key tensors.

    Args:
        q (`torch.Tensor`): The query tensor.
        k (`torch.Tensor`): The key tensor.
        cos (`torch.Tensor`): The cosine part of the rotary embedding.
        sin (`torch.Tensor`): The sine part of the rotary embedding.
        position_ids (`torch.Tensor`, *optional*):
            Deprecated and unused.
        unsqueeze_dim (`int`, *optional*, defaults to 1):
            The 'unsqueeze_dim' argument specifies the dimension along which to unsqueeze cos[position_ids] and
            sin[position_ids] so that they can be properly broadcasted to the dimensions of q and k. For example, note
            that cos[position_ids] and sin[position_ids] have the shape [batch_size, seq_len, head_dim]. Then, if q and
            k have the shape [batch_size, heads, seq_len, head_dim], then setting unsqueeze_dim=1 makes
            cos[position_ids] and sin[position_ids] broadcastable to the shapes of q and k. Similarly, if q and k have
            the shape [batch_size, seq_len, heads, head_dim], then set unsqueeze_dim=2.
    Returns:
        `tuple(torch.Tensor)` comprising of the query and key tensors rotated using the Rotary Position Embedding.
    """
    cos = cos.unsqueeze(unsqueeze_dim)
    sin = sin.unsqueeze(unsqueeze_dim)
    q_embed = (q * cos) + (rotate_half(q) * sin)
    k_embed = (k * cos) + (rotate_half(k) * sin)
    return q_embed, k_embed

def rotate_half(x):
    """Rotates half the hidden dims of the input."""
    x1 = x[..., : x.shape[-1] // 2]
    x2 = x[..., x.shape[-1] // 2 :]
    return torch.cat((-x2, x1), dim=-1)

def repeat_kv(hidden_states: torch.Tensor, n_rep: int) -> torch.Tensor:
    """
    This is the equivalent of torch.repeat_interleave(x, dim=1, repeats=n_rep). The hidden states go from (batch,
    num_key_value_heads, seqlen, head_dim) to (batch, num_attention_heads, seqlen, head_dim)
    """
    batch, num_key_value_heads, slen, head_dim = hidden_states.shape
    if n_rep == 1:
        return hidden_states
    hidden_states = hidden_states[:, :, None, :, :].expand(batch, num_key_value_heads, n_rep, slen, head_dim)
    return hidden_states.reshape(batch, num_key_value_heads * n_rep, slen, head_dim)



In [3]:
from pag.utils import get_module_by_name

# monkey patch forward path
import math
import os
import warnings
from dataclasses import dataclass
from typing import List, Optional, Tuple, Union

from transformers.modeling_outputs import (
    CausalLMOutputWithPast,
)

def forward(
    self,
    input_ids: torch.LongTensor = None,
    attention_mask: Optional[torch.Tensor] = None,
    position_ids: Optional[torch.LongTensor] = None,
    past_key_values: Optional[List[torch.FloatTensor]] = None,
    inputs_embeds: Optional[torch.FloatTensor] = None,
    labels: Optional[torch.LongTensor] = None,
    use_cache: Optional[bool] = None,
    output_attentions: Optional[bool] = None,
    output_hidden_states: Optional[bool] = None,
    return_dict: Optional[bool] = None,
    cache_position: Optional[torch.LongTensor] = None,
) -> Union[Tuple, CausalLMOutputWithPast]:
    r"""
    Args:
        labels (`torch.LongTensor` of shape `(batch_size, sequence_length)`, *optional*):
            Labels for computing the masked language modeling loss. Indices should either be in `[0, ...,
            config.vocab_size]` or -100 (see `input_ids` docstring). Tokens with indices set to `-100` are ignored
            (masked), the loss is only computed for the tokens with labels in `[0, ..., config.vocab_size]`.

    Returns:

    Example:

    ```python
    >>> from transformers import AutoTokenizer, LlamaForCausalLM

    >>> model = LlamaForCausalLM.from_pretrained("meta-llama/Llama-2-7b-hf")
    >>> tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf")

    >>> prompt = "Hey, are you conscious? Can you talk to me?"
    >>> inputs = tokenizer(prompt, return_tensors="pt")

    >>> # Generate
    >>> generate_ids = model.generate(inputs.input_ids, max_length=30)
    >>> tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
    "Hey, are you conscious? Can you talk to me?\nI'm not conscious, but I can talk to you."
    ```"""
    global guidance_scale, hook_layer_list
    
    output_attentions = output_attentions if output_attentions is not None else self.config.output_attentions
    output_hidden_states = (
        output_hidden_states if output_hidden_states is not None else self.config.output_hidden_states
    )
    return_dict = return_dict if return_dict is not None else self.config.use_return_dict

    ########
    # base #
    ########
    # decoder outputs consists of (dec_features, layer_state, dec_hidden, dec_attn)
    outputs = self.model(
        input_ids=input_ids,
        attention_mask=attention_mask,
        position_ids=position_ids,
        past_key_values=past_key_values,
        inputs_embeds=inputs_embeds,
        use_cache=use_cache,
        output_attentions=output_attentions,
        output_hidden_states=output_hidden_states,
        return_dict=return_dict,
        cache_position=cache_position,
    )

    hidden_states = outputs[0]
    if self.config.pretraining_tp > 1:
        lm_head_slices = self.lm_head.weight.split(self.vocab_size // self.config.pretraining_tp, dim=0)
        logits_o = [F.linear(hidden_states, lm_head_slices[i]) for i in range(self.config.pretraining_tp)]
        logits_o = torch.cat(logits_o, dim=-1)
    else:
        logits_o = self.lm_head(hidden_states)
    logits_o = logits_o.float()

    #######
    # PAG #
    #######
    # 1. hook (make attn map identity)
    handles = []
    for layer_name in hook_layer_list:
        module = get_module_by_name(self, layer_name)
        handle = module.register_forward_hook(perturb_attn_map, with_kwargs=True)
        handles.append(handle)

    # 2. compute logit 
    # decoder outputs consists of (dec_features, layer_state, dec_hidden, dec_attn)
    outputs = self.model(
        input_ids=input_ids,
        attention_mask=attention_mask,
        position_ids=position_ids,
        past_key_values=past_key_values,
        inputs_embeds=inputs_embeds,
        use_cache=use_cache,
        output_attentions=output_attentions,
        output_hidden_states=output_hidden_states,
        return_dict=return_dict,
        cache_position=cache_position,
    )

    hidden_states = outputs[0]
    if self.config.pretraining_tp > 1:
        lm_head_slices = self.lm_head.weight.split(self.vocab_size // self.config.pretraining_tp, dim=0)
        logits_p = [F.linear(hidden_states, lm_head_slices[i]) for i in range(self.config.pretraining_tp)]
        logits_p = torch.cat(logits_p, dim=-1)
    else:
        logits_p = self.lm_head(hidden_states)
    logits_p = logits_p.float()

    # remove hook
    for handle in handles:
        handle.remove()

    # 3. guidance
    logits = logits_o + guidance_scale * (logits_o - logits_p)

    loss = None
    if not return_dict:
        output = (logits,) + outputs[1:]
        return (loss,) + output if loss is not None else output

    return CausalLMOutputWithPast(
        loss=loss,
        logits=logits,
        past_key_values=outputs.past_key_values,
        hidden_states=outputs.hidden_states,
        attentions=outputs.attentions,
    )


# monkey patch (ref;  https://discuss.pytorch.org/t/monkey-patching-the-forward-pass-of-an-nn-module/176095)
# old_forward = model.forward
import types
pipe.model.forward = types.MethodType(forward, pipe.model)

In [11]:
hook_layer_list = [
    # 'model.layers.0.self_attn',
    # 'model.layers.1.self_attn',
    # 'model.layers.2.self_attn',
    # 'model.layers.3.self_attn',
    # 'model.layers.4.self_attn',
    # 'model.layers.5.self_attn',
    # 'model.layers.6.self_attn',
    # 'model.layers.7.self_attn',
    # 'model.layers.8.self_attn',
    # 'model.layers.9.self_attn',
    # 'model.layers.10.self_attn',
    # 'model.layers.11.self_attn',
    # 'model.layers.12.self_attn',
    'model.layers.13.self_attn',
    'model.layers.14.self_attn',
    'model.layers.15.self_attn',
    'model.layers.16.self_attn',
    'model.layers.17.self_attn',
    'model.layers.18.self_attn',
    'model.layers.19.self_attn',
    'model.layers.20.self_attn',
    # 'model.layers.21.self_attn',
    # 'model.layers.22.self_attn',
    # 'model.layers.23.self_attn',
    # 'model.layers.24.self_attn',
    # 'model.layers.25.self_attn',
    # 'model.layers.26.self_attn',
    # 'model.layers.27.self_attn',
    # 'model.layers.28.self_attn',
    # 'model.layers.29.self_attn',
    # 'model.layers.30.self_attn',
    # 'model.layers.31.self_attn',
]
import numpy as np

guidance_scale_list = np.linspace(0, 4, 5)
for guidance_scale in guidance_scale_list:
    prompt = "264 + 117"
    # prompt = "# python code for adding two numbers \n\n def "
    outputs = pipe(prompt, max_length=64, truncation=True)
    print()
    print(f'### {round(guidance_scale, 2)} ###')
    print(outputs[0]['generated_text'])
    print()

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



### 0.0 ###
264 + 117 117
2012-03-23 2012-03-24 2012 2013 2014 2014 2015 2018 2016 2015  2015 2016 2014 2013 2014  2018



Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



### 1.0 ###
264 + 117 = 0 + 0. In the case of the latter, the number of zeros is 1. The number of zeros in the first column is 1 + 2 and the number of ones is 1 in the second column. The number of zeros in the second is zero columns is



Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



### 2.0 ###
264 + 117 = 0 + 0. In the 2010s, she has been married to her husband's partner in life.



Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



### 3.0 ###
264 + 117 =
+  +  +  + ------>   +    + => => => replace     =
+  +                  =>      =>    replace        =>    replace replace replace replace replace replace replace replace replace replace replace replace replace replace replace replace replace replace replace replace replace replace replace replace replace replace


### 4.0 ###
264 + 117 =
+  + = volume times volume :
+ + + strength + + stronger salt stronger sugar stronger spices spices spices spices spices spices spices spices spices spices spices spices spices spices spices spices spices spices spices spices spices spices spices spices spices spices spices spices spices spices spices spices spices spices spices spices spices spices spices spices



### PAG 

In [9]:
hook_layer_list = [
    # 'transformer.h.0.attn', 
    # 'transformer.h.1.attn', 
    'transformer.h.2.attn', 
    'transformer.h.3.attn', 
    'transformer.h.4.attn', 
    'transformer.h.5.attn', 
    # 'transformer.h.6.attn', 
    # 'transformer.h.7.attn', 
    # 'transformer.h.8.attn', 
    # 'transformer.h.9.attn', 
    # 'transformer.h.10.attn', 
    # 'transformer.h.11.attn', 
]

tokenizer.pad_token_id = tokenizer.eos_token_id
for guidance_scale in range(15):
    guidance_traj = []

    inputs = tokenizer(["Today is"], return_tensors="pt")
    outputs = model.generate(**inputs, max_new_tokens=15, return_dict_in_generate=True, output_scores=True)

    # generated result
    print()
    print(f"### Generated text - guidance_scale {guidance_scale} ###")
    print(f'{tokenizer.decode(outputs.sequences[0], skip_special_tokens=True)}')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



### Generated text - guidance_scale 0 ###
Today is the day when we can all be proud of our country and our values.


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



### Generated text - guidance_scale 1 ###
Today is Election Day and the Republican Party is in a state of flux. The party


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...


### Generated text - guidance_scale 11 ###
Today is Election Eve and Jill continue her historic opposition to unaccountability in politics today afternoon


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



### Generated text - guidance_scale 12 ###
Today is Election Season when integrity advocates stockpile stinkiolDOSDOSEmpty(), stinkivia


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



### Generated text - guidance_scale 13 ###
Today is Election Season when integrity advocates stockpile stinkiolDOS executable's withoutifying Unifiedavis

### Generated text - guidance_scale 14 ###
Today is Election Season when integrity advocates stockpile stinkiolDOS executable's withoutifying Unifiedavis


In [10]:
# Perplexity 계산 함수
# def calculate_perplexity(model_original, model_modified, tokenizer, text, guidance_scale):
#     encoded_text = tokenizer.encode(text, return_tensors='pt')
#     attention_mask = torch.ones_like(encoded_text)
    
#     with torch.no_grad():
#         outputs_original = model_original(encoded_text, attention_mask=attention_mask, labels=encoded_text)
#         loss_original = outputs_original.loss
        
#         outputs_modified = model_modified(encoded_text, attention_mask=attention_mask)
#         logits_modified = outputs_modified.logits
        
#         logits = outputs_original.logits + guidance_scale * (outputs_original.logits - logits_modified)
#         loss_modified = torch.nn.functional.cross_entropy(logits.view(-1, logits.size(-1)), encoded_text.view(-1))
        
#         perplexity = torch.exp((loss_original + loss_modified) / 2)
    
#     return perplexity.item()

In [11]:
# Perplexity 계산 및 출력
# perplexity = calculate_perplexity(model_original, model_modified, tokenizer, generated_text, guidance_scale)
# print(f"Perplexity: {perplexity:.2f}")